# Data Science di Cloud: Cara "Azure ML SDK"

## Pendahuluan

Dalam notebook ini, kita akan belajar cara menggunakan Azure ML SDK untuk melatih, menerapkan, dan menggunakan model melalui Azure ML.

Prasyarat:
1. Anda telah membuat workspace Azure ML.
2. Anda telah memuat [dataset Heart Failure](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) ke dalam Azure ML.
3. Anda telah mengunggah notebook ini ke Azure ML Studio.

Langkah-langkah berikutnya adalah:

1. Membuat sebuah Eksperimen di Workspace yang sudah ada.
2. Membuat sebuah kluster Compute.
3. Memuat dataset.
4. Mengonfigurasi AutoML menggunakan AutoMLConfig.
5. Menjalankan eksperimen AutoML.
6. Mengeksplorasi hasil dan mendapatkan model terbaik.
7. Mendaftarkan model terbaik.
8. Menerapkan model terbaik.
9. Menggunakan endpoint.

## Impor khusus Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Inisialisasi Workspace
Inisialisasi objek workspace dari konfigurasi yang tersimpan. Pastikan file konfigurasi tersedia di .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Membuat eksperimen Azure ML

Mari kita buat eksperimen bernama 'aml-experiment' di workspace yang baru saja kita inisialisasi.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Membuat Compute Cluster
Anda perlu membuat [compute target](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) untuk menjalankan AutoML Anda.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Data
Pastikan Anda telah mengunggah dataset ke Azure ML dan bahwa kunci memiliki nama yang sama dengan dataset.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Konfigurasi AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Jalankan AutoML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Men-deploy Model Terbaik

Jalankan kode berikut untuk men-deploy model terbaik. Anda dapat melihat status deployment di portal Azure ML. Langkah ini mungkin memerlukan beberapa menit.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Menggunakan Endpoint
Anda dapat menambahkan input ke contoh input berikut.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan terjemahan yang akurat, harap diingat bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
